## Number 6
Picking up from the discussion of simultaneous equations above, where y is N x k, and $y = X\beta{}\ + u$  
If X is N x l and cov(u|X) = $\Omega{}\$ then this is a generalization of the assumption of homoskedasticity to a multivariate setting; the resulting structure is called a system of Seemingly Unrelated Regressions (SUR).

(i) If $\Omega{}\$ isn't diagonal then there's a sense in which the different
equations in the system are dependent, since observing a realization of, say, y1 may change our prediction of y2. (This is why the system is called seemingly unrelated.) Describe this dependence formally.

If omega is not diagonal and the equations in the system are dependent, that means the errors are correlated across equations. To tackle this problem, we may consider estimation by Generalized Least Squares (GLS) which we will premultiply by $\Omega^{^-1/2}$ so that the transformed vector is independent and identically distributed. 

$y=\bar{X}\cdot\beta{}+e$

$E[e|X] = 0$

$E[ee'|X] = \Omega{}\$

$\hat{\beta}{_g}{_l}{_s} = (\sum_{i=1}^{n}\bar{X_i}'\sum^{-1}\bar{X_i})^{-1}(\sum_{i=1}^{n}\bar{X_i}'\sum^{-1}{Y_i})^{-1}$

Since $\Omega\$ is unknown, it must be replaced by an estimator 

$E[ee'|X] = \hat{\Omega}\$

Using $\hat{\Omega}\$, we obtain a feasible GLS estimator

$\hat{\beta}{_s}{_u}{_r} = (\sum_{i=1}^{n}\bar{X_i}'\hat{\Omega^{-1}}\bar{X_i})^{-1}(\sum_{i=1}^{n}\bar{X_i}'\hat{\Omega^{-1}}{Y_i})^{-1}$

$ = (\bar{X_i}{'}(I_n\cdot\hat{\Omega^{-1}}\bar{X})^{-1}(\bar{X_i}{'}(I_n\cdot\hat{\Omega^{-1}}Y)$

(ii) Adapt the code in weighted_regression.ipynb so that the data-generating process for u can accommodate a general covariance matrix such as $\Omega{}$, and let X = T. Estimate $\beta{}$.

In [47]:
# Import packages
%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal
from scipy.linalg import inv, sqrtm

In [48]:
# Set seed
import random
random.seed(1234)

In [49]:
# Create random variables 
k = 3 # Number of observables in T

mu = [0]*k

Sigma=[[1,0.5,0],
       [0.5,2,0],
       [0,0,3]]

T = multivariate_normal(mu,Sigma)

Omega = [[1,0.2],
         [0.2,2]]  # covariance matrix for u

u = multivariate_normal(mean=[0,0],cov=Omega)

In [50]:
# Define X and construct a sample
beta = [1/2,1]

D = np.random.random(size=(3,2)) # Generate random 3x2 matrix

N=1000 # Sample size

# Now: Transform rvs into a sample
T = T.rvs(N)

u = u.rvs(N) # Replace u with a sample

X = (T**3)@D  # Note use of ** operator for exponentiation

y = X@beta + u[:,0] # Note use of @ operator for matrix multiplication

In [51]:
# Estimate B
Omega_inv = inv(Omega)
W = np.zeros((N, N))
for i in range(N):
    W[i,i] = Omega_inv[0,0]
b = np.linalg.lstsq(T.T @ W @ X, T.T @ W @ y, rcond=None)[0] # lstsqs returns several results

e = y - X@b

print(b)

TXplus = np.linalg.pinv(T.T@ W @ X) # Moore-Penrose pseudo-inverse

# Covariance matrix of b
vb = e.var()*TXplus@T.T@ W @ W @ T@TXplus.T  # u is known to be homoskedastic

print(vb)

[0.49544231 0.99470168]
[[ 9.26048771e-05 -7.78176508e-05]
 [-7.78176508e-05  9.17867687e-05]]


(iii) How are the estimates obtained from this SUR system different from what one would obtain if one estimated equation by equation using OLS?

The estimator, Omega-hat can be updated by calculating the SUR residuals and the covariance matrix. Substitued, we obtain an iterated SUR estimator  that can be iterated until convergence. 